In [1]:
from glob import glob as gg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import rc as rc
#
from netCDF4 import Dataset  

In [21]:
#
# mesh information
#
class mesh_data:
    def __init__(self, filename):
        self.read_vertical_mesh(filename)
        
    def read_vertical_mesh(self,filename):
        ncid = Dataset(filename, "r", format="NETCDF4")        
        
        # read dates 
        self.dates = ncid.variables['dates'][:]
        # read vertical mesh
        self.nlevels, self.zbar = ncid.variables['nlevels'][:], ncid.variables['zbar'][:]
        
        ncid.close()

In [22]:
#
# read mesh information
#
path_mesh = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/grid/'
filename = path_mesh + 'REcoM1D_mesh.nc'
mesh = mesh_data(filename)

In [23]:
#
# initial constant conditions
#
PAR_init = 50         # in W/m2
Kz_init = 0.01        # in m2/s
T_init = 1            # in degree Celsius
S_init = 34           # in ppt
aice_init = 90        # in %
#shortwave_init = 50   # in W/m2

In [24]:
#
# initialize arrays
#
# ice cover
aice = np.zeros(nt) + aice_init/100
# temperature, salinity, light and turbulence
nlvl, nt = len(mesh.zbar), len(mesh.nlevels)
init = np.zeros((nlvl, nt)) * np.nan
T, S, PAR, Kz = np.copy(init), np.copy(init), np.copy(init), np.copy(init)

for i in range(nt):
    ind=int(mesh.nlevels[i])
    T[:ind,i], S[:ind,i] = T_init, S_init
    PAR[:ind,i], Kz[:,ind] = PAR_init, Kz_init 

In [30]:
#
# save forcing to netcdf
#
class Output(object):
    def __init__(self, filename, dates, aice, T, S, PAR, Kz):
        self.write_recom_constant_forcing(filename, dates, aice, T, S, PAR, Kz)
        
    def write_recom_constant_forcing(self, filename, dates,aice, T, S, PAR, Kz):
        
        nt, nl = len(dates), PAR.shape[0]
        
        # store grid (Lagrangian and vertical) information
        ncid = Dataset(filename, "w", format="NETCDF4") 

        ## define dimensionss
        ncid.createDimension('time', nt)       
        ncid.createDimension('level', nl)
        dim = ('level', 'time')

        ## create variables
        # dates
        dt = ncid.createVariable('dates', "f8",'time')
        
        # ice cover
        ice = ncid.createVariable('aice', "f8",'time')
                             
        # temperature and salinity
        t, s  = ncid.createVariable('temperature', "f8", dim), ncid.createVariable('salinity', "f8",dim)
        
        # vertical mesh
        par, kz  = ncid.createVariable('PAR', "f8",dim), ncid.createVariable('Kz', "f8",dim)
                
        
        ## variables attributes
        # description
        dt.description='dates related to python ordinal dates (reference date 01/01/0001)'
        ice.description = 'surface of ocean covered by ice'
        t.description, s.description = 'temperature distribution in the water column',  'salinity distribution the water column'
        par.description, kz.description = 'Photosynthetically Active Radiation in the water column', 'turbulence in the water column'
        # units
        dt.units='days'
        ice.units, t.units, s.units = '', 'degree (Celsius)', 'ppt'
        par.units, kz.units = 'W/m2', 'm2/s'       
        
        ## fill variables
        # dates
        ncid['dates'][:] = dates
        
        # ice
        ncid['aice'][:] = aice
        
        print(T.shape, S.shape)
        # temperature and salinity
        ncid['temperature'][:] , ncid['salinity'][:] = T, S
        
        # radiation and turbulence
        ncid['PAR'][:] , ncid['Kz'][:] = PAR, Kz
                             
        #
        ncid.close()
#
#
#
path_forcing = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/forcing/'
filename = path_forcing + 'REcoM1D_forcing.nc'
Output(filename, mesh.dates, aice, T, S, PAR, Kz)

(48, 7201) (48, 7201)
